In [ ]:
import sys
import os
# sys.path.append("../doctr/")
# from doctr.models import ocr_predictor
# from doctr.io import DocumentFile
import json
import pickle
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

In [ ]:
def get_file_by_dict(json_output):
    '''
    Given json output from OCR, construct a dictionary to better represent the data
    '''
    file_as_dict = {'words':[], 'ymin_ymax':[], 'page_id':[], 'page_dimensions':[], 'line_merge_next':[], 'bboxes': []}
    for page in json_output['pages']:
        for block in page['blocks']:
            for idx, line in enumerate(block['lines']):
                ((xmin, ymin),(xmax, ymax)) = line['geometry']

                
                file_as_dict['words'].append([line['words'][i]['value'] for i in range(len(line['words']))])
                file_as_dict['bboxes'].append([line['words'][ii]['geometry'] for ii in range(len(line['words']))])
                
                file_as_dict['ymin_ymax'].append((ymin, ymax))
                file_as_dict['page_id'].append(page['page_idx'])
                file_as_dict['page_dimensions'].append(page['dimensions'])

    return file_as_dict

def getIOU(segment1, segment2, threshold):
    '''
    Merging algorithm for line geometries. If lines are in different blocks but have similar line coordinates, we will be 
    able to "merge" them as one line this way. Function needed because OCR model does not always treat the same line
    as an item within the same block
    '''
    ymin1, ymax1 = segment1
    ymin2, ymax2 = segment2
    
    less_ymax = min(ymax1, ymax2)
    less_ymin = min(ymin1, ymin2)
    greater_ymax = max(ymax1, ymax2)
    greater_ymin = max(ymin1, ymin2)
    
    intersection = less_ymax - greater_ymin
    
    # no overlap
    if intersection < 0:
        return False
    
    union = greater_ymax - less_ymin
    
    if intersection / union > threshold:
        return True
    return False

def get_lines_to_merge(file_as_dict, threshold):
    lines_to_merge_indices = []
    for i in range(0, len(file_as_dict['words'])):
        for j in range(i+1, len(file_as_dict['words'])):
            if (file_as_dict['page_id'][i] == file_as_dict['page_id'][j]) and getIOU(file_as_dict['ymin_ymax'][i], 
                                                                                     file_as_dict['ymin_ymax'][j], 
                                                                                     threshold):
                lines_to_merge_indices.append((i,j))
    return lines_to_merge_indices

def line_merging(lines_to_merge_indices):
    follows = dict()
    lines = dict()
    for i, j in lines_to_merge_indices:

        if i not in lines:
            if i not in follows:
                lines[i] = [i, j]
                follows[j] = [i]
            else:  
                one_link_back = follows[i][0]
                while one_link_back in follows:
                    one_link_back = follows[one_link_back][0]
                if j not in lines[one_link_back]:
                    lines[follows[i][0]].append(j)
        else:
            lines[i].append(j)

        if j not in follows:
            follows[j] = [i]
        else:
            follows[j].append(i)
    return lines, follows

In [ ]:
def final_file_line_by_line(file_as_dict, threshold):
    final_file_as_dict = {'full_line':[], 'page_id':[], 'ymax_max':[], 'ymin_min':[], 'bboxes': []}
    lines_to_merge = get_lines_to_merge(file_as_dict, threshold)
    line_merge_map, follow_merge_map = line_merging(lines_to_merge)

    # final_file_as_dict['bboxes'] = [bbox for bbox in file_as_dict['bboxes']]
    for i in range(len(file_as_dict['words'])):
        if i not in line_merge_map and i not in follow_merge_map:
            final_file_as_dict['full_line'].extend([file_as_dict['words'][i]])
            final_file_as_dict['bboxes'].extend([file_as_dict['bboxes'][i]])
            final_file_as_dict['page_id'].append(file_as_dict['page_id'][i])
            final_file_as_dict['ymin_min'].append(file_as_dict['ymin_ymax'][i][0])
            final_file_as_dict['ymax_max'].append(file_as_dict['ymin_ymax'][i][1])
            
        else:
            if i in line_merge_map:
                line = []
                ymin_min = []
                ymax_max = []
                bboxes = []
                for j in line_merge_map[i]:
                    line.extend(file_as_dict['words'][j])
                    ymin_min.append(file_as_dict['ymin_ymax'][j][0])
                    ymax_max.append(file_as_dict['ymin_ymax'][j][1])
                    bboxes.extend(file_as_dict['bboxes'][j])
                
                final_file_as_dict['full_line'].append(line)
                final_file_as_dict['bboxes'].append(bboxes)
                final_file_as_dict['ymin_min'].append(ymin_min)
                final_file_as_dict['ymax_max'].append(ymax_max)
                final_file_as_dict['page_id'].append(file_as_dict['page_id'][i])
                
                #final_file_as_dict['full_line'].append([file_as_dict['words'][j] for j in line_merge_map[i]])
                #final_file_as_dict['ymin_min'].append([file_as_dict['ymin_ymax'][j][0] for j in line_merge_map[i]])
                #final_file_as_dict['ymax_max'].append([file_as_dict['ymin_ymax'][j][1] for j in line_merge_map[i]])
    
    
    for idx, (min_element, max_element) in enumerate(zip(final_file_as_dict['ymin_min'], final_file_as_dict['ymax_max'])):
        if type(min_element) == type(list()):
            new_min_element = min(min_element)
            new_max_element = max(max_element)
            final_file_as_dict['ymin_min'][idx] = new_min_element
            final_file_as_dict['ymax_max'][idx] = new_max_element
    
    print(len(final_file_as_dict['full_line']), len(final_file_as_dict['bboxes']))
    return final_file_as_dict

def get_toc_page(preprocessed_output):
    regex_exp = r"(table of contents|table*of*(?:contents)?|(?:table\s)?of*conten|contents?)"
    for page_id, line in zip(preprocessed_output['page_id'],
                             preprocessed_output['full_line']):
        
        ref_str = " ".join(line).lower()
        matches = re.search(regex_exp, ref_str)
        if matches is not None:
            print(f"page_id: {page_id}")
            return page_id
    return None

def find_start(section_info, preprocessed_output,
               subset_match_threshold, line_len_match_threshold,
               beg_line_match_threshold, last_line_pointer):
    '''
    Given a toc section title, iterate all the lines in the file from the last line associated with a section title
    going forward
    '''
    
    if last_line_pointer == len(preprocessed_output['full_line']):
        return None, last_line_pointer
    
    toc_page = get_toc_page(preprocessed_output)
    
    for idx in range(last_line_pointer, len(preprocessed_output['full_line'])):
        
        if preprocessed_output['page_id'][idx] == toc_page:
            continue
        line = " ".join(preprocessed_output['full_line'][idx])
        bboxes = preprocessed_output['bboxes'][idx]
    
        beg_line = line[0: len(section_info[0])*2]

        if (fuzz.partial_ratio(section_info[0].lower(), line.lower()) > subset_match_threshold and
            len(line) >= len(section_info[0]) * line_len_match_threshold                       and
            fuzz.partial_ratio(section_info[0].lower(), beg_line.lower()) > beg_line_match_threshold):
            
            ymin = preprocessed_output['ymin_min'][idx]
            ymax = preprocessed_output['ymax_max'][idx]
            page_id = preprocessed_output['page_id'][idx]
            
            return (line, section_info[0], ymin, ymax, page_id, bboxes), idx + 1
    
    return None, last_line_pointer

def get_starts_all(section_dict, preprocessed_output):
    last_line_pointer = 0
    #lines = list(zip(preprocessed_output2['page_id'],
    #                  preprocessed_output2['full_line'],
    #                  preprocessed_output2['ymin_min'],
    #                  preprocessed_output2['ymax_max']))


    subset_match_threshold = 80    # Design decision to only allow subset match ratios of > 80/100
    line_len_match_threshold = 0.8 # Design decision to potentially only match document lines
                                   # that are not much smaller than table of content label
    beg_line_match_threshold = 80
    starts = []
    toc_page = get_toc_page(preprocessed_output)

    for key, section_info in section_dict.items():

        start, last_line_pointer = find_start(section_info, preprocessed_output,
                                               subset_match_threshold, line_len_match_threshold,
                                               beg_line_match_threshold, last_line_pointer)
        if start:
            starts.append(start)
        else:
            print(f"Couldn't match {section_info[0]} with a line. Moving onto next TOC section")
    
    return starts
def flatten_contract_dict(nested_dict):
    i = 1
    section_dict_flattened = {}

    for item in nested_dict.items():
        section_dict_flattened[i] = (item[1][0], {})
        i += 1
        for sub_item in item[1][1].items():
            section_dict_flattened[i] = (sub_item[1][0], {})
            i += 1
    return section_dict_flattened

In [ ]:
def merge_bboxes(words, query, bboxes, window_size, match_score):
    
    max_window_score, match_idx = 0, 0
    for idx, start_idx in enumerate(range(0, len(words), window_size)):
        window_text = " ".join(words[start_idx : start_idx + window_size])
        
        window_score = process.extractBests(window_text, [query], scorer=fuzz.token_set_ratio)[0][-1]
        candidate_bboxes = bboxes[start_idx : start_idx + window_size]

        if window_score >= match_score:
            # x min is the x_left of the first bbox 
            # y_min is the min of the top left y's for each box
            x_min = candidate_bboxes[0][0][0]
            y_min = min([y[0][-1] for y in candidate_bboxes])

            # x max is the x_right of the last bbox 
            # y_max is the max of the bottom_right y's for each box
            x_max = candidate_bboxes[-1][1][0]
            y_max = max([y[1][-1] for y in candidate_bboxes])

            merged_bbox = [ [x_min, y_min], [x_max, y_max] ]
            
            return merged_bbox, window_text
        
        else:
            if window_score > max_window_score:
                max_window_score = window_score
                match_text = window_text
                match_candidate_bboxes = candidate_bboxes

            continue
    
    # in case the window_text has a lower match score than with the entire string, match with the 
    # segment with highest matching score
    x_min = match_candidate_bboxes[0][0][0]
    y_min = min([y[0][-1] for y in match_candidate_bboxes])

    # x max is the x_right of the last bbox 
    # y_max is the max of the bottom_right y's for each box
    x_max = match_candidate_bboxes[-1][1][0]
    y_max = max([y[1][-1] for y in match_candidate_bboxes])

    merged_bbox = [ [x_min, y_min], [x_max, y_max] ]

    return merged_bbox, match_text

def extract_exact_match(row):
    
    query = row['Section Title via HTML']
    if query is not None:

        match = process.extractBests(row.loc['Line via OCR'], [query], scorer=fuzz.token_set_ratio)
        
        match_text, match_score = match[0]
        window_size = len(match_text.split())
        
        
        merged_bbox, exact_match_text = merge_bboxes(row['Line via OCR'].split(), 
                                                     query, 
                                                     row['bboxes'], 
                                                     window_size, 
                                                     match_score)
        
        return merged_bbox, exact_match_text
    
    else:
        return None, None

In [ ]:
# toc_page_id = 1

In [ ]:
with open("TOC_Labels_Set1.pkl", "rb") as f:
    section_dicts = pickle.load(f)
section_dicts = [flatten_contract_dict(section_dict) for section_dict in section_dicts]

In [ ]:
json_files = {
    'pdf_0': 
            {
                "name": "ZogenixInc_20190509_10-Q_EX-10.2_11663313_EX-10.2_Distributor Agreement",
                "path": "pdf_0_from_list_in_discord.json"
            },
    'pdf_1': 
            {
                "name": "PerformanceSportsBrandsInc_20110909_S-1_EX-10.10_7220214_EX-10.10_Endorsement Agreement",
                "path": "pdf_1_from_list_in_discord.json"
            },
    'pdf_2': 
            {
                "name": "OTISWORLDWIDECORP_04_03_2020-EX-10.4-INTELLECTUAL PROPERTY AGREEMENT by and among UNITED TECHNOLOGIES CORPORATION, OTIS WORLDWIDE CORPORATION and CARRIER ~1",
                "path": "pdf_2_from_list_in_discord.json"
            },
    'pdf_3': 
            {
                "name": "NUVEEN - REMARKETING AGREEMENT",
                "path": "pdf_3_from_list_in_discord.json"
            },
    'pdf_4': 
            {
                "name": "'ParatekPharmaceuticalsInc_20170505_10-KA_EX-10.29_10323872_EX-10.29_Outsourcing Agreement",
                "path": "pdf_4_from_list_in_discord.json"
            },
 

}

## Let's see the outputs now for all files

In [ ]:
file_key = 'pdf_4'
json_path = json_files[file_key]['path']
filename = json_files[file_key]['name']
pickle_key = int(file_key.split('_')[1])
section_dict = section_dicts[pickle_key]


with open(json_path, 'r') as f:
    json_output = json.load(f)
file_as_dict = get_file_by_dict(json_output)
preprocessed_output = final_file_line_by_line(file_as_dict, threshold=0.65)
df = pd.DataFrame(preprocessed_output)
preprocessed_output = df.sort_values(by=['page_id', 'ymin_min']).to_dict(orient='list')

df = pd.DataFrame(get_starts_all(section_dict, preprocessed_output)).rename(columns={0:'Line via OCR',
                                     1:'Section Title via HTML',
                                     2:'ymin',
                                     3:'ymax',
                                     4:'page_id',
                                     5:'bboxes'})
df['exact_match_bbox'], df['exact_match_text'] = zip(*df.apply(lambda row: extract_exact_match(row), axis=1))
df

Looks like NUVEEN - REMARKETING AGREEMENT is a perfect match

## Big concern for bias in labeling data: Will many contract section headers be biased to these type of contracts where the header is normal font sized and usually in line with other words?

On to the next example

Looks like it matches the table of contents again...let's confirm

Oh wait no it actually matched the lines! It just looked like TOC matches but it was not.

## Out of 5 articles here are results:
1. PerformanceSportsBrandsInc_20110909_S-1_EX-10.10_7220214_EX-10.10_Endorsement Agreement
    - perfect match

2. ZogenixInc_20190509_10-Q_EX-10.2_11663313_EX-10.2_Distributor Agreement 
    - many good matches; doesn't match lines where TOC label is actually 2 lines in the document (specifically cases where the label is "ARTICLE X ...."

3. 'OTISWORLDWIDECORP_04_03_2020-EX-10.4-INTELLECTUAL PROPERTY AGREEMENT by and among UNITED TECHNOLOGIES CORPORATION, OTIS WORLDWIDE CORPORATION and CARRIER ~1'
    - matched the TOC page itself. Need better method to find that page. Use state dictionary and match twice method OR fuzzy matching OR some other method
    
4. NUVEEN - REMARKETING AGREEMENT i
    - Perfect match
    
5. ParatekPharmaceuticalsInc_20170505_10-KA_EX-10.29_10323872_EX-10.29_Outsourcing Agreement
    - Looks like a perfect match except for these two lines in the actual TOC that are actually ambiguous. Not sure what they even mean. The two lines were:
        - 3.8 [* * *]
        - 3.9 [* * * ]

## Merge code 11-10-22

In [ ]:
'''
toc_page_id = df.loc[df['Line via OCR'].str.contains(r'table of contents|tableof|table of', case=False), 'page_id'] 
if not toc_page_id.empty:
    toc_page_id = toc_page_id.iloc[0] 

toc_page_id
'''

return back to above later and integrate regex